<a href="https://colab.research.google.com/github/GVSU-CIS635/term-project-CDC-Diabetes/blob/Association-Rules/Assoc_rules_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.10.12


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import matplotlib.pyplot as plt

# Read the clean data file
clean_data = pd.read_csv("/content/drive/MyDrive/cdc_2015.csv")
display(clean_data)

Mounted at /content/drive


,Diabetes_012,HighChol,Smoker,MentHlth,Education,Income
0,0,1,1,18,4,3
1,0,0,1,0,6,1
2,0,1,0,30,4,8
3,0,0,0,0,3,6
4,0,1,0,3,5,4
...,...,...,...,...,...,...
282364,2,1,0,0,2,4
282365,0,0,0,0,5,2
282366,0,1,1,20,4,5
282367,0,0,0,0,5,1


In [ ]:
# Create the dataframe
clean_df= pd.DataFrame(clean_data)

# Create the columns for the model
clean_df['combined_column'] = ''

def concatenate_columns(row):
   """
   Concatenate the values in the 'Name', 'Age',
   and 'Country' columns with a separator of '|'.
   """
   return str(row['HighChol']) + '|' + str(row['Smoker']) + '|' + str(row['MentHlth']) + '|' + str(row['Education']) + '|' + str(row['Income'])

# Apply the function to each row of the DataFrame
clean_df['combined_column'] = clean_df.apply(concatenate_columns, axis=1)

display(clean_df)

,Diabetes_012,HighChol,Smoker,MentHlth,Education,Income,combined_column
0,0,1,1,18,4,3,1|1|18|4|3
1,0,0,1,0,6,1,0|1|0|6|1
2,0,1,0,30,4,8,1|0|30|4|8
3,0,0,0,0,3,6,0|0|0|3|6
4,0,1,0,3,5,4,1|0|3|5|4
...,...,...,...,...,...,...,...
282364,2,1,0,0,2,4,1|0|0|2|4
282365,0,0,0,0,5,2,0|0|0|5|2
282366,0,1,1,20,4,5,1|1|20|4|5
282367,0,0,0,0,5,1,0|0|0|5|1


In [ ]:
# This is to isolate the Diabetes positive group

outliers = [0,2]
clean_df_1 = clean_df[clean_df.Diabetes_012.isin(outliers) == False]
display(clean_df_1)

,Diabetes_012,HighChol,Smoker,MentHlth,Education,Income,combined_column
80,1,1,0,30,4,3,1|0|30|4|3
104,1,1,0,3,4,7,1|0|3|4|7
229,1,1,0,0,4,8,1|0|0|4|8
282,1,1,1,30,6,6,1|1|30|6|6
323,1,1,1,0,5,7,1|1|0|5|7
...,...,...,...,...,...,...,...
282200,1,0,1,0,6,7,0|1|0|6|7
282238,1,0,0,0,6,4,0|0|0|6|4
282325,1,1,0,30,4,3,1|0|30|4|3
282350,1,1,1,0,6,4,1|1|0|6|4


In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

clean_df_test=clean_df_1[['Diabetes_012','HighChol','Smoker']]

# Building the model
frq_items = apriori(clean_df_test, min_support = 0.05, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

specified_consequent = {'Diabetes_012'}

filtered_rules = rules[rules['consequents'] == specified_consequent]
#print(rules.head())
display(filtered_rules)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(HighChol),(Diabetes_012),0.620696,1.0,0.620696,1.0,1.0,0.0,inf,0.0
2,(Smoker),(Diabetes_012),0.492867,1.0,0.492867,1.0,1.0,0.0,inf,0.0
6,"(Smoker, HighChol)",(Diabetes_012),0.329465,1.0,0.329465,1.0,1.0,0.0,inf,0.0


In [1]:
# This is to isolate the Diabetes negative group

outliers = [1,2]
clean_df_0 = clean_df[clean_df.Diabetes_012.isin(outliers) == False]
display(clean_df_0)

# Association rules for Diabetes negative group
clean_df_test0=clean_df_0[['Diabetes_012','HighChol','Smoker']]

# Building the model
frq_items_0 = apriori(clean_df_test0, min_support = 0.05, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules_0 = association_rules(frq_items_0, metric ="lift", min_threshold = 1)
rules_0 = rules_0.sort_values(['confidence', 'lift'], ascending =[False, False])

specified_consequent = {'Diabetes_012'}

filtered_rules_0 = rules_0[rules_0['consequents'] == specified_consequent]
#print(rules.head())
display(filtered_rules_0)

NameError: name 'clean_df' is not defined